In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import glob
import mysql.connector
from mysql.connector import Error

In [3]:
query_analytics = 'SELECT * FROM analytics_perf ap WHERE Account = "SB" AND landingPagePath like "%/de/%"'
query_lightspeed = 'SELECT * FROM api_lightspeed_saintbasics_orders WHERE Shipping_Country = "DE"'
query_adwords = 'SELECT aaas.Observation, aac.Name, SUM(aaas.Impr) AS "impressions", SUM(aaas.Clicks) AS "clicks", SUM(aaas.Cost) AS "cost" FROM api_adwords.api_adwords_ag_stats aaas JOIN api_adwords.api_adwords_adgroups aaa on aaa.ID = aaas.Adgroup_ID JOIN api_adwords.api_adwords_campaigns aac on aac.ID = aaa.Campaign_ID JOIN api_adwords.api_adwords_accounts aaa2 on aaa2.ID = aaa.Account_ID WHERE aaa2.ID = "6109709000" GROUP BY 1,2;'
query_facebook = 'SELECT * FROM api_facebook.account_report WHERE Account_ID = "395756991466104"'

In [4]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_analytics',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_ga = pd.read_sql(query_analytics,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_analytics',)
MySQL connection is closed


In [66]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_lightspeed',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_ls = pd.read_sql(query_lightspeed,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_lightspeed',)
MySQL connection is closed


In [6]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_adwords',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_aw = pd.read_sql(query_adwords,con=connection)
        df_fb = pd.read_sql(query_facebook,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_adwords',)
MySQL connection is closed


**Transformations**

In [7]:
df_ga['Observation'] = pd.to_datetime(df_ga['Observation'])
df_ga['year_week'] = df_ga.Observation.dt.isocalendar()['year'].astype(str)+"-"+df_ga.Observation.dt.isocalendar()['week'].astype(str) #all
df_ga['year_month'] = df_ga.Observation.dt.strftime("%Y-%m")

In [8]:
df_ls['Conv_Date'] = pd.to_datetime(df_ls['Conv_Date'])
df_ls['year_week'] = df_ls.Conv_Date.dt.isocalendar()['year'].astype(str)+"-"+df_ls.Conv_Date.dt.isocalendar()['week'].astype(str)
df_ls['year_month'] = df_ls.Conv_Date.dt.strftime("%Y-%m")
df_ls['Conv_Type'] = df_ls['Conv_Type'].fillna('not_returned')

pat = r"^((?!not_returned).)*$"
df_ls['Conv_Type'] = df_ls['Conv_Type'].str.replace(pat, 'returned').str.replace('exchange\r\n\r\nverzenden met DHL!', 'returned')
df_ls = df_ls.groupby(['Conv_Date', 'Conv_ID', 'Conv_Type', 'Hashed_Email', 'year_week', 'year_month']).agg({'Revenue':pd.Series.mean}).reset_index()

In [67]:
df_ls['Conv_Date'] = pd.to_datetime(df_ls['Conv_Date'])
df_ls['year_week'] = df_ls.Conv_Date.dt.isocalendar()['year'].astype(str)+"-"+df_ls.Conv_Date.dt.isocalendar()['week'].astype(str)
df_ls['year_month'] = df_ls.Conv_Date.dt.strftime("%Y-%m")
df_ls['Conv_Type'] = df_ls['Conv_Type'].fillna('not_returned')

pat = r"^((?!not_returned).)*$"
df_ls['Conv_Type'] = df_ls['Conv_Type'].str.replace(pat, 'returned').str.replace('exchange\r\n\r\nverzenden met DHL!', 'returned')

In [72]:
df_ls['Conv_Type'].unique()

array(['not_returned', 'returned'], dtype=object)

In [71]:
df_ls.tail(30)

,Account,Market,Conv_Date,Conv_ID,SKU,Conv_Number,Conv_Type,Revenue,Revenue_excl_VAT,Customer_ID,Hashed_Email,Shipping_Country,Shipping_Amount,Status,year_week,year_month
2076,SB,DE,2021-01-08,172638641,UT040100171,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2077,SB,DE,2021-01-08,172638641,UT080100171,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2078,SB,DE,2021-01-08,172638641,WB06050014,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2079,SB,DE,2021-01-08,172638641,WT08020014,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2080,SB,DE,2021-01-08,172638641,WT09010014,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2081,SB,DE,2021-01-08,172638641,WT10050014,2007447,not_returned,184.00,184.00,629307,a3a091d14747c0d0649e59dfd70d72f9,DE,0.00,None,2021-1,2021-01
2082,SB,DE,2021-01-09,172658085,MB05090014,2007448,not_returned,102.40,102.40,629307,59d77e9486355adbca412073af8106f2,DE,0.00,None,2021-1,2021-01
2083,SB,DE,2021-01-09,172658085,WB09050014,2007448,not_returned,102.40,102.40,629307,59d77e9486355adbca412073af8106f2,DE,0.00,None,2021-1,2021-01
2084,SB,DE,2021-01-09,172658085,WB09150014,2007448,not_returned,102.40,102.40,629307,59d77e9486355adbca412073af8106f2,DE,0.00,None,2021-1,2021-01
2085,SB,DE,2021-01-09,172658085,WB09250014,2007448,not_returned,102.40,102.40,629307,59d77e9486355adbca412073af8106f2,DE,0.00,None,2021-1,2021-01


In [9]:
df_aw['Observation'] = pd.to_datetime(df_aw['Observation'])
df_aw['year_week'] = df_aw.Observation.dt.isocalendar()['year'].astype(str)+"-"+df_aw.Observation.dt.isocalendar()['week'].astype(str)
df_aw['year_month'] = df_aw.Observation.dt.strftime("%Y-%m")

In [10]:
df_fb['Observation'] = pd.to_datetime(df_fb['Observation'])
df_fb['year_week'] = df_fb.Observation.dt.isocalendar()['year'].astype(str)+"-"+df_fb.Observation.dt.isocalendar()['week'].astype(str)
df_fb['year_month'] = df_fb.Observation.dt.strftime("%Y-%m")

**Aggregations**

In [58]:
ga_summary_week_all = df_ga.groupby(['year_week']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_week_channels = df_ga.groupby(['year_week','channelGrouping']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_week_all['cvr'] = ga_summary_week_all['transactions']/ga_summary_week_all['sessions']
ga_summary_week_channels['cvr'] = ga_summary_week_channels['transactions']/ga_summary_week_channels['sessions']

ga_summary_week_all.rename(columns={'transactionRevenue': 'ga_transactionRevenue', 'sessions':'ga_sessions', 'transactions': 'ga_transactions'}, inplace=True)

In [59]:
to_drop1 = ga_summary_week_channels[ga_summary_week_channels['transactions'] == 0].index
ga_summary_week_channels.drop(index=to_drop1, inplace=True)
ga_summary_week_channels.reset_index(drop=True, inplace=True)
ga_summary_week_channels.head(2)

,year_week,channelGrouping,transactionRevenue,sessions,transactions,cvr
0,2020-45,Direct,111.10,53,1,0.018868
1,2020-45,Email,23.40,2,1,0.500000
2,2020-46,Direct,36.00,40,1,0.025000
3,2020-46,Referral,194.00,4,1,0.250000
4,2020-46,SEO,74.15,63,2,0.031746
5,2020-47,Direct,64.85,51,1,0.019608
6,2020-47,Email,170.75,5,1,0.200000
7,2020-47,SEO,143.10,109,2,0.018349
8,2020-48,Direct,96.30,61,2,0.032787
9,2020-48,Email,104.45,6,1,0.166667


In [60]:
ga_summary_month_all = df_ga.groupby(['year_month']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_month_channels = df_ga.groupby(['year_month','channelGrouping']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_month_all['cvr'] = ga_summary_month_all['transactions']/ga_summary_month_all['sessions']
ga_summary_month_channels['cvr'] = ga_summary_month_channels['transactions']/ga_summary_month_channels['sessions']

ga_summary_month_all.rename(columns={'transactionRevenue': 'ga_transactionRevenue', 'sessions':'ga_sessions', 'transactions': 'ga_transactions'}, inplace=True)

In [61]:
to_drop = ga_summary_month_channels[ga_summary_month_channels['transactions'] == 0].index
ga_summary_month_channels.drop(index=to_drop, inplace=True)
ga_summary_month_channels.reset_index(drop=True, inplace=True)
ga_summary_month_channels

,year_month,channelGrouping,transactionRevenue,sessions,transactions,cvr
0,2020-11,Direct,405.20,216,6,0.027778
1,2020-11,Email,298.60,18,3,0.166667
2,2020-11,Referral,194.00,21,1,0.047619
3,2020-11,SEO,696.82,346,9,0.026012
4,2020-12,Direct,799.35,471,5,0.010616
5,2020-12,Email,476.05,108,5,0.046296
6,2020-12,Referral,219.55,88,2,0.022727
7,2020-12,SEA - Brand,122.05,117,2,0.017094
8,2020-12,SEO,1143.70,262,10,0.038168
9,2021-01,Direct,112.50,203,1,0.004926


In [14]:
ls_summary_week_to = df_ls.groupby(['year_week','Conv_Type']).agg({'Revenue':pd.Series.sum}).unstack(1).fillna(0) #'Conv_ID'
ls_summary_week_tx = df_ls.groupby(['year_week','Conv_Type']).agg({'Conv_ID':pd.Series.nunique}).unstack(1).fillna(0)
ls_summary_week_to.columns = ls_summary_week_to.columns.droplevel()
ls_summary_week_tx.columns = ls_summary_week_tx.columns.droplevel()
ls_summary_week_to.columns = ['TO_EXRET','TO_RET']
ls_summary_week_tx.columns = ['TX_EXRET','TX_RET']
ls_summary_week = ls_summary_week_to.merge(ls_summary_week_tx, on='year_week')


In [65]:
df_ls.tail()

,Conv_Date,Conv_ID,Conv_Type,Hashed_Email,year_week,year_month,Revenue
668,2021-01-09,172696397,not_returned,98ff028ddace88b0d7def57d93bd7872,2021-1,2021-01,35.75
669,2021-01-10,172744775,not_returned,c82d6fd390b609898ebb8aac8912f3f9,2021-1,2021-01,292.00
670,2021-01-10,172786081,not_returned,3e514429ad676de39cdaa74fbd61c6f3,2021-1,2021-01,74.55
671,2021-01-12,172968031,not_returned,11582f7479b97593b2f1644e1a300770,2021-2,2021-01,42.95
672,2021-01-13,173135893,not_returned,3f0b367b33aab05223286df06c8d9773,2021-2,2021-01,40.55


In [64]:
ls_summary_week

,TO_EXRET,TO_RET,TX_EXRET,TX_RET
year_week,,,,
2019-40,526.05,0.0,6.0,0.0
2019-41,1086.00,0.0,16.0,0.0
2019-42,655.00,0.0,9.0,0.0
2019-43,602.75,0.0,8.0,0.0
2019-44,793.22,0.0,9.0,0.0
...,...,...,...,...
2020-7,342.10,0.0,4.0,0.0
2020-8,706.80,0.0,8.0,0.0
2020-9,901.11,87.9,10.0,2.0


In [15]:
ls_summary_month_to = df_ls.groupby(['year_month','Conv_Type']).agg({'Revenue':pd.Series.sum}).unstack(1).fillna(0)
ls_summary_month_tx = df_ls.groupby(['year_month','Conv_Type']).agg({'Conv_ID':pd.Series.nunique}).unstack(1).fillna(0)
ls_summary_month_to.columns = ls_summary_month_to.columns.droplevel()
ls_summary_month_tx.columns = ls_summary_month_tx.columns.droplevel()
ls_summary_month_to.columns = ['TO_EXRET','TO_RET'] #Revenue
ls_summary_month_tx.columns = ['TX_EXRET','TX_RET']
ls_summary_month = ls_summary_month_to.merge(ls_summary_month_tx, on='year_month')
ls_summary_month.tail()

,TO_EXRET,TO_RET,TX_EXRET,TX_RET
year_month,,,,
2020-09,3347.67,295.35,35.0,4.0
2020-10,3702.31,24.95,41.0,1.0
2020-11,2906.62,72.95,33.0,2.0
2020-12,3560.52,240.00,33.0,4.0
2021-01,1990.30,287.70,16.0,4.0


In [16]:
aw_summary_week = df_aw.groupby(['year_week']).agg({'impressions':pd.Series.sum, 'clicks':pd.Series.sum, 'cost':pd.Series.sum}).reset_index()
aw_summary_week.rename(columns={'impressions': 'impressions_aw', 'clicks':'clicks_aw', 'cost': 'cost_aw'}, inplace=True)
aw_summary_month = df_aw.groupby(['year_month']).agg({'impressions':pd.Series.sum, 'clicks':pd.Series.sum, 'cost':pd.Series.sum}).reset_index()
aw_summary_month.rename(columns={'impressions': 'impressions_aw', 'clicks':'clicks_aw', 'cost': 'cost_aw'}, inplace=True)

In [17]:
fb_summary_week = df_fb.groupby(['year_week']).agg({'Impr':pd.Series.sum, 'Reach':pd.Series.sum, 'Frequency':pd.Series.sum, 'Clicks':pd.Series.sum, 'Cost':pd.Series.sum}).reset_index()
fb_summary_week.rename(columns={'Impr': 'impressions_fb', 'Clicks':'clicks_fb', 'Cost': 'cost_fb'}, inplace=True)
fb_summary_month = df_fb.groupby(['year_month']).agg({'Impr':pd.Series.sum, 'Reach':pd.Series.sum, 'Frequency':pd.Series.sum, 'Clicks':pd.Series.sum, 'Cost':pd.Series.sum}).reset_index()
fb_summary_month.rename(columns={'Impr': 'impressions_fb', 'Clicks':'clicks_fb', 'Cost': 'cost_fb'}, inplace=True)

**Summaries**

In [73]:
monthly_summary = ls_summary_month.merge(ga_summary_month_all, on='year_month')
monthly_summary = monthly_summary.merge(aw_summary_month, on='year_month')
monthly_summary = monthly_summary.merge(fb_summary_month, on='year_month')
monthly_summary['total_cost'] = monthly_summary['cost_fb'] + monthly_summary['cost_aw']
monthly_summary['ROAS'] = monthly_summary['TO_EXRET']/monthly_summary['total_cost']
monthly_summary['AOV'] = monthly_summary['TO_EXRET']/( monthly_summary['TX_EXRET']+monthly_summary['TX_RET'])
monthly_summary['CPO'] = monthly_summary['total_cost']/( weekly_summary['TX_EXRET']+weekly_summary['TX_RET'])
monthly_summary['CVR_ls'] =( monthly_summary['TX_EXRET']+monthly_summary['TX_RET']) /monthly_summary['ga_sessions']
#monthly_summary['CPS'] = monthly_summary['total_cost']/monthly_summary['ga_sessions']
monthly_summary.columns = monthly_summary.columns.str.upper()
monthly_summary

,YEAR_MONTH,TO_EXRET,TO_RET,TX_EXRET,TX_RET,GA_TRANSACTIONREVENUE,GA_SESSIONS,GA_TRANSACTIONS,CVR,IMPRESSIONS_AW,...,IMPRESSIONS_FB,REACH,FREQUENCY,CLICKS_FB,COST_FB,TOTAL_COST,ROAS,AOV,CPO,CVR_LS
0,2020-12,3560.52,240.0,33.0,4.0,2760.7,7312,24,0.003282,1805.0,...,214977,162671,210.269,11769,2399.758,2764.378,1.288000,96.23027,394.911143,0.005060
1,2021-01,1990.30,287.7,16.0,4.0,664.0,1477,6,0.004062,675.0,...,63682,52551,98.657,2207,724.570,882.950,2.254148,99.51500,73.579167,0.013541


In [74]:
weekly_summary = ls_summary_week.merge(ga_summary_week_all, on='year_week')
weekly_summary = weekly_summary.merge(aw_summary_week, on='year_week')
weekly_summary = weekly_summary.merge(fb_summary_week, on='year_week')
weekly_summary['total_cost'] = weekly_summary['cost_fb'] + weekly_summary['cost_aw']
weekly_summary['ROAS'] = weekly_summary['TO_EXRET']/weekly_summary['total_cost']
weekly_summary['AOV'] = weekly_summary['TO_EXRET']/( weekly_summary['TX_EXRET']+weekly_summary['TX_RET'])
weekly_summary['CPO'] = weekly_summary['total_cost']/( weekly_summary['TX_EXRET']+weekly_summary['TX_RET'])
weekly_summary['CVR_ls'] =( weekly_summary['TX_EXRET']+weekly_summary['TX_RET']) /weekly_summary['ga_sessions']
weekly_summary.columns = weekly_summary.columns.str.upper()
weekly_summary

,YEAR_WEEK,TO_EXRET,TO_RET,TX_EXRET,TX_RET,GA_TRANSACTIONREVENUE,GA_SESSIONS,GA_TRANSACTIONS,CVR,IMPRESSIONS_AW,...,IMPRESSIONS_FB,REACH,FREQUENCY,CLICKS_FB,COST_FB,TOTAL_COST,ROAS,AOV,CPO,CVR_LS
0,2020-50,1190.90,0.0,7.0,0.0,854.20,2066,5,0.002420,548.0,...,63719,47558,55.850,4177,803.060,905.890,1.314619,170.128571,129.412857,0.003388
1,2020-51,1078.60,80.5,11.0,1.0,1004.50,2177,10,0.004593,597.0,...,60078,44686,59.119,2854,720.519,834.459,1.292574,89.883333,69.538250,0.005512
2,2020-52,455.05,0.0,5.0,0.0,253.50,2278,3,0.001317,404.0,...,63416,48696,62.832,3596,542.159,640.029,0.710983,91.010000,128.005800,0.002195
3,2020-53,1332.27,56.9,11.0,2.0,309.30,1179,3,0.002545,376.0,...,50032,39867,56.562,1898,577.400,673.460,1.978247,102.482308,51.804615,0.011026
4,2021-1,1046.85,287.7,9.0,4.0,508.55,738,4,0.005420,368.0,...,31246,25972,52.130,1084,354.350,437.330,2.393730,80.526923,33.640769,0.017615
5,2021-2,83.50,0.0,2.0,0.0,155.45,273,2,0.007326,169.0,...,10168,8443,22.433,367,126.840,155.990,0.535291,41.750000,77.995000,0.007326


In [20]:
from gspread_pandas import Spread, Client

In [21]:
saintb_mt_sheet = Spread('SAINTBASICS_MT_DATA')

In [75]:
# Push Transactions
saintb_mt_sheet.df_to_sheet(monthly_summary, index=True, sheet='Monthly_Summary', start='A1', replace=True)

In [76]:
# Push Transactions
saintb_mt_sheet.df_to_sheet(weekly_summary, index=True, sheet='Weekly_Summary', start='A1', replace=True)

In [62]:
# Push Transactions
saintb_mt_sheet.df_to_sheet(ga_summary_month_channels, index=True, sheet='GA_Monthly', start='A1', replace=True)

In [63]:
# Push Transactions
saintb_mt_sheet.df_to_sheet(ga_summary_week_channels, index=True, sheet='GA_Weekly', start='A1', replace=True)